In [ ]:
!pip install torch accelerate transformers langchain chromadb sentence-transformers rank_bm25

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving caf_chunks.json to caf_chunks.json


In [ ]:
!pip install pyngrok streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.7 MB/s eta 0:00:00


In [ ]:
!streamlit run app.py &

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [ ]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers langchain chromadb sentence-transformers rank_bm25 pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers langchain langchain-community chromadb sentence-transformers rank_bm25 pyngrok

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import json
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_community.vectorstores import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.chains import RetrievalQA

In [ ]:
def flatten_metadata(metadata):
    flattened = {}
    for k, v in metadata.items():
        if isinstance(v, (dict, list)):
            flattened[k] = json.dumps(v, ensure_ascii=False)
        else:
            flattened[k] = v
    return flattened

In [ ]:
# ✅ 3️⃣ Upload CAF chunks (caf_chunks.json)
from google.colab import files
uploaded = files.upload()  # select 'caf_chunks.json' from your PC

with open("caf_chunks.json", "r") as f:
    chunks = json.load(f)

texts = [c["content"] for c in chunks]
metadatas = [flatten_metadata(c["metadata"]) for c in chunks]

Saving caf_chunks.json to caf_chunks.json


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma.from_texts(
    texts,
    embeddings,
    metadatas=metadatas,
    collection_name="caf_collection",
    persist_directory="./chroma_db"
)
db.persist()

/tmp/ipython-input-2384294829.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-2384294829.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [ ]:
model_name = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=False,      # greedy decoding
    max_new_tokens=200,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=generator)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-2060242856.py:22: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator)


In [ ]:
bm25_retriever = BM25Retriever.from_texts(texts, metadatas)
dense_retriever = db.as_retriever(search_kwargs={"k": 8})

hybrid_retriever = EnsembleRetriever(
    retrievers=[dense_retriever, bm25_retriever],
    weights=[0.6, 0.4]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=hybrid_retriever
)

In [ ]:
system_prompt = """
You are a cybersecurity assistant for Cyberland Park.
Answer clearly, concisely, and only once.
Do not repeat phrases.
Include relevant MITRE techniques with their names and IDs if possible.
"""

def build_prompt(user_question, context=""):
    return f"{system_prompt}\nContext:\n{context}\nUser question: {user_question}"

In [ ]:
print("Cyberland Park Cybersecurity Chatbot (type 'exit' to quit)")
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    # build the prompt (no undefined variables)
    prompt = build_prompt(query)

    # invoke the QA chain
    response = qa.invoke(prompt)

    # handle dict or string outputs safely
    if isinstance(response, dict) and "result" in response:
        answer = response["result"]
    else:
        answer = str(response)

    print("\n--- Cyberland Park Assistant ---")
    print(answer.strip())
    print("-------------------------------\n")

Cyberland Park Cybersecurity Chatbot (type 'exit' to quit)
You: How can we avoid data theft in our research center department?

--- Cyberland Park Assistant ---
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

You have defined appropriate cyber security training and awareness activities for all roles in your organisation, from executives to the most junior roles.

All people in your organisation understand the contribution they make to the essential functions cyber security.

You use a range of teaching and communication techniques for cyber security training and awareness to reach the widest audience effectively.

You make cyber security information and good practice guidance easily accessible, widely available and you know it is referenced and used within your organisation.

You routinely evaluate your cyber security training and awareness activities to ensure they reach

KeyboardInterrupt: Interrupted by user